# Signing and verifying a block header 

## 1. Creating Encryption Key pair

In [1]:
def generate_RSA(bits=2048):
    '''
    Generate an RSA keypair with an exponent of 65537 in PEM format
    param: bits The key length in bits
    Return private key and public key
    '''
    from Crypto.PublicKey import RSA 
    new_key = RSA.generate(bits, e=65537) 
    public_key = new_key.publickey().exportKey("PEM") 
    private_key = new_key.exportKey("PEM") 
    return private_key, public_key


In [2]:
from Crypto.PublicKey import RSA 

alice_priv, alice_pub = generate_RSA()
bob_priv, bob_pub = generate_RSA()

with open("resources/alice_pub.pem", "wb") as file:
    file.write(alice_pub)

with open("resources/alice_priv.pem", "wb") as file:
    file.write(alice_priv)

with open("resources/bob_pub.pem", "wb") as file:
    file.write(bob_pub)

with open("resources/bob_priv.pem", "wb") as file:
    file.write(bob_priv)

## 2. Signing a blocker header using private key

In [5]:
import json 
from Crypto.Hash import SHA

def hash_block(msg=""):
    # Hashes a list of dictionaries
    if type(msg)!=str:
        msg = json.dumps(msg,sort_keys=True)  # If we don't sort keys, we can't guarantee repeatability!
        
    return SHA.new(str.encode(msg))

In [7]:
from Crypto.PublicKey import RSA

# Load resources
with open("resources/alice_priv.pem", "rb") as file:
    alice_priv = RSA.importKey(file.read())
    
with open('resources/alice_pub.pem', "rb") as file:
    alice_pub = RSA.importKey(file.read())

with open("resources/bob_priv.pem", "rb") as file:
    bob_priv = RSA.importKey(file.read())
    
with open('resources/bob_pub.pem', "rb") as file:
    bob_pub = RSA.importKey(file.read())

In [12]:
from Crypto.Signature import PKCS1_v1_5

txns = [
    {"Bob": -50, "Alice": 50},
    {"Bob": -25, "Alice": 25}
]

block_header = {u'blockNumber':0, u'parentHash':None, u'txn':txns}
hash_obj = hash_block(block_header)
signer = PKCS1_v1_5.new(bob_priv)
signature = signer.sign(hash_obj)

signature

b'_w\x10\xca\x18U\x14>\xa8\xfa8\xd4tQ=D7]\x18\xff\x8d\xd7vKK`!`\x11\xdb\x1c\xd1\xc5\xd0Z\xad\x91\xb50\x142\xcdBr6\xf7\x1e>ROW^\xdfW\\\x1d\xb7\x9f\xd7e\xea\xce\xad\xa7\xd5\xd6\x0bN\xb4\x96b\xa3$V65c\x07\x7f.J\x14\x04W\xb9@\x12zA\xde\xa4D\x844I\x871\xfb\x05_\xf98\xc5Q\xb7\xdf\x0f\xe5\x13\xc7\xb1\xb5\x93nkm\xa9I\x90\xc7@if\xa1\x14%v\x14\x0e%{\xf2\xd9\xc7\xb9a]\x99\xabS\xc2oO\x07\xb2\xe8\x0cc"\x85r\xdf\r\xc0"\xcfhcV\xf8@8\x0b\xc2\xaaHaq\xbf\x03\xc50\xc5Vc\x9ff\xe9\xd1\x15\x9a\x04\xbbM\x88\xdc\xfa\x89\xfb\xab\x8a+Ef;\xa2D\x07Jz\xa4\x1f\xcdP\x90\x8ff\xf5\xf8\xd4\x0e\xf0\xb8\xec\x1aC]\x7f\xa4\xb0f\xea\x81\x12\xf9i6\x07\xce4\x89\x8e\x7f\xa2\x15\x08PU/\x1fB\xd3Sd\x84q]\x0c\x99$+\xce\xcb\xbc\x00\xc5'

In [13]:
import pickle

with open("resources/signature.pkl", "wb") as file: 
    pickle.dump(signature, file)

## 3. Verifying a block header using public key

In [14]:
from Crypto.Signature import PKCS1_v1_5
import pickle

with open("resources/signature.pkl", "rb") as file: 
    signature = pickle.load(file)

txns = [
    {"Bob": -50, "Alice": 50},
    {"Bob": -25, "Alice": 25}
]

block_header = {u'blockNumber':0, u'parentHash':None, u'txn':txns}
hash_obj = hash_block(block_header)

verifier = PKCS1_v1_5.new(bob_pub)

result = verifier.verify(hash_obj, signature)

result

True